In [4]:
import ctypes
import cv2
import numpy as np
import time

In [2]:
ekmi = ctypes.cdll.LoadLibrary('./../ekmi.so')
_get_ekmi = ekmi.get_ekmi
_get_ekmi.argtypes = [
    ctypes.c_int, 
    np.ctypeslib.ndpointer(dtype=np.uintp, flags="C", ndim=1), 
    ctypes.c_int,
    ctypes.c_double
]
_get_ekmi.restype = ctypes.POINTER(ctypes.POINTER(ctypes.c_double))


def get_ekmi(order, img, p):
    imgpp = (img.__array_interface__['data'][0] + np.arange(img.shape[0])* img.strides[0]).astype(np.uintp) 
    # run ekmi
    tik = time.time()
    _res = _get_ekmi(order, imgpp, img.shape[0], p)
    tok = time.time()
    print(tok - tik)
    # copy array
    res = np.zeros((order, order))
    for i in range(order):
        for j in range(order):
            res[i, j] = _res[i][j]
    ekmi.free_ekmi_rst() # free memory
    return res

In [61]:
order = 15
p = 0.5

fname = "/Users/kx/desktop/1.png"
img = cv2.imread(fname)
img = cv2.resize(img, (64, 64))
img = (img[:, :, 0] > 0).astype('int32')


get_ekmi(order, img, p)

1.9495911598205566


array([[  1.        ,  -0.45454188,  -1.86112781,  -3.22385385,
         -4.54631535,  -5.8316861 ,  -7.08278203,  -8.30211321,
         -9.4919265 , -10.65424095, -11.79087749, -12.90348376,
        -13.9935553 , -15.06245346, -16.11142088],
       [  0.83224452,  -0.62204852,  -2.02848505,  -3.39115174,
         -4.71363577,  -5.9991038 ,  -7.25036547,  -8.46992531,
         -9.66002522, -10.82267989, -11.95970626, -13.07274847,
        -14.16329884, -15.23271586, -16.28223953],
       [  0.66866118,  -0.7854138 ,  -2.19172789,  -3.55435863,
         -4.87688532,  -6.16246765,  -7.41390923,  -8.63370917,
         -9.82410462, -10.98710601, -12.1245265 , -13.23800678,
        -14.32903611, -15.3989702 , -16.44904652],
       [  0.50929451,  -0.94459029,  -2.35080618,  -3.71342189,
         -5.03600902,  -6.32172056,  -7.57335421,  -8.7934039 ,
         -9.98410215, -11.14745528, -12.28527278, -13.39919203,
        -14.49069934, -15.56114771, -16.61177221],
       [  0.35415418,  -1.09

In [108]:
import glob
import os
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from sklearn.neighbors import KNeighborsClassifier

In [62]:
order = 16
p = 0.5

In [63]:
data_dir = '/Users/kx/Docs/github/git_cccr_sl/chinese-calligraphy-recognition-sl/data/shufadict/clean'
chars = [x for x in os.listdir(data_dir) if x[0] != '.']
N = 64
nclass = 10
sample_per_class = 10

In [64]:
class_lookup = [None] * nclass
data = np.zeros((nclass * sample_per_class, N, N))
label = np.zeros((nclass * sample_per_class), dtype='int')
feature = np.zeros((nclass * sample_per_class, order, order))

In [65]:
for i, char in enumerate(chars[:nclass]):
    class_lookup[i] = char
    fnames = glob.glob('{}/{}/*.png'.format(data_dir, char))[:sample_per_class]
    for j, fname in enumerate(fnames):
        img = cv2.imread(fname)
        img = cv2.resize(img, (N, N))
        img = (img[:, :, 0] > 0).astype('int32')
        data[i * nclass + j, :, :] = img
        label[i * nclass + j] = i

In [66]:
for i in tqdm(range(nclass * sample_per_class)):
    img = data[i, :, :]
    feature[i, :, :] = get_ekmi(order, img, p)

  0%|          | 0/100 [00:00<?, ?it/s]

1.376471996307373
1.3500969409942627
1.360914945602417
1.3588275909423828
1.3700690269470215
1.3770129680633545
1.3690979480743408
1.37105393409729
1.3759329319000244
1.3549540042877197
1.351313829421997
1.352977991104126
1.3575949668884277
1.4721829891204834
1.4356369972229004
1.3977890014648438
1.419234037399292
1.3748371601104736
1.3903021812438965
1.390489101409912
1.3772258758544922
1.3820972442626953
1.3907828330993652
1.3818669319152832
1.387930154800415
1.3939497470855713
1.3929181098937988
1.3865020275115967
1.3940048217773438
1.4095618724822998
1.4539799690246582
1.516780138015747
1.4343838691711426
1.4105720520019531
1.4004738330841064
1.3785381317138672
1.3774049282073975
1.3782310485839844
1.3771028518676758
1.3831679821014404
1.3885929584503174
1.3840129375457764
1.387450933456421
1.4055919647216797
1.3738539218902588
1.387639045715332
1.3950510025024414
1.3983840942382812
1.4178552627563477
1.3885042667388916
1.3864898681640625
1.3811330795288086
1.3917210102081299
1.401

In [102]:
test_idxs = np.concatenate([[i * nclass + j for i in range(nclass)] for j in range(2)])
test_idxs.sort()
train_idxs = [x for x in range(train.shape[0]) if x not in test_idxs]

test = train[test_idxs]
test_label = label[test_idxs]
test_feature = feature[test_idxs]

train = train[train_idxs]
train_label = label[train_idxs]
train_feature = feature[train_idxs]

In [117]:
X = train.reshape((train.shape[0], train.shape[1] * train.shape[2]))
y = test.reshape((test.shape[0], test.shape[1] * test.shape[2]))

In [138]:
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X, train_label)
sum(knn.predict(y) == test_label) / len(test_label)

0.9

In [150]:
dist = np.zeros((train.shape[0], train.shape[0]))
for i in range(train.shape[0]):
    for j in range(train.shape[0]):
        dist[i, j] = sum(sum((feature[i, :, :] - feature[j, :, :]) ** 2))
        
np.where((dist > 0) & (dist < 1))

In [174]:
9927 / 2 / 60

82.725

In [170]:
len(glob.glob('{}/*/*.png'.format(data_dir)))

9927